After a first approach to the data analysis, my main ideas are:

- feature engineering on timestamp to extract relevant info (like period, month, season but also time, night, etc)
- managing of source... one hot encoding? Can explode with that amount of different value_counts but as a first approach may be adequate
- the most important part: title and content
  - title: tfidf vectorizer + post process to remove some columns
  - content: for now, the first approach, it's better to not use the content. It's quite a mess, but the fact that is an HTML page can be very helpful for the structure and extracting structured and hierarchical infos. But for now, let's remove it
- pagerank simply treated as continuos, because the order makes sense so let's keep it as is

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from src.preprocessing import Preprocessor
import pandas as pd

prep = Preprocessor()
prep.full_prep()

vectorizer = TfidfVectorizer(stop_words='english', min_df=10)
tfidf_matrix = vectorizer.fit_transform(prep.df['title'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [2]:
(prep.df.shape[0] - (tfidf_df == 0).sum()).sort_values(ascending=False)


ap             8347
reuters        3427
39             3411
afp            2808
new            2734
               ... 
sidelined        10
advertisers      10
advisers         10
addicts          10
admit            10
Length: 6511, dtype: int64

In [3]:
tfidf_df.shape[1]


6511

In [4]:
import numpy as np 
import re 

pattern = r'^[0-9].+$'
matches = [
    m[0]
    for word in vectorizer.get_feature_names_out()
    if (m := re.findall(pattern, word))
]
len(matches)


155

In [5]:
tfidf_df.drop(columns=matches, inplace=True, errors='ignore')


In [6]:
with open('cols.txt', 'w') as f:
    for col in tfidf_df.columns:
        f.write(col +'\n')

In [7]:
tfidf_df.drop(columns=tfidf_df.columns[(tfidf_df.columns.str.len() <= 2)], inplace=True)
len(tfidf_df.columns)


6262

In [8]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=0.95)

# out = pca.fit(tfidf_df)

In [9]:
# pca.transform(tfidf_df)

In [10]:
tfidf_df

,a380,abandoned,abbas,abbey,abc,abducted,abduction,abe,able,abn,...,zero,zeta,ziff,zimbabwe,zombie,zone,zoo,zuma,zune,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
prep.df

,source,title,article,page_rank,timestamp,label
Id,,,,,,
0,AllAfrica.com,OPEC Boosts Nigeria&#39;s Oil Revenue By .82m Bpd,THE Organisation of Petroleum Exporting Countr...,5,2004-09-16 22:39:53,5
1,Xinhua,Yearender: Mideast peace roadmap reaches dead-...,Looking back at the major events that took pla...,5,2004-12-17 19:01:14,0
2,Yahoo,Battleground Dispatches for Oct. 5 \\n (CQP...,CQPolitics.com - Here are today's Battleground...,5,2006-10-05 18:42:29,0
3,BBC,Air best to resuscitate newborns,Air rather than oxygen should be used to resus...,5,<NA>,0
4,Yahoo,High tech German train crash kills at least on...,"<p><a href=""http://us.rd.yahoo.com/dailynews/r...",5,2006-09-22 17:28:57,0
...,...,...,...,...,...,...
79992,Yahoo,Italy's embattled Prodi faces vote of confiden...,"<p><a href=""http://us.rd.yahoo.com/dailynews/r...",5,2008-01-23 11:39:35,0
79993,All-Baseball.com,"Ding Dong, the Deal is Dead","As yesterday began, there was widespread antic...",5,<NA>,4
79994,Yahoo,Two bombs discovered in Sardinia after Berlusc...,AFP - Police discovered two bombs near the Sar...,5,<NA>,0
